# O Colab irá reiniciar nessa primeira etapa. Isso é normal.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!wget https://data.qiime2.org/distro/core/qiime2-2021.2-py36-linux-conda.yml
!conda env create -n qiime2-2021.2 --file qiime2-2021.2-py36-linux-conda.yml -qq

# Daqui em diante, é possível executar tudo de uma vez.

In [ ]:
%%bash
git clone https://github.com/Gnoogit/metagen.git

In [ ]:
%%shell

wget -O "patient_joao_MICROBIOMA16S_S69_R1_001.fastq.gz" "https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/TCC-metagenomica/patient_joao_MICROBIOMA16S_S69_R1_001.fastq.gz"
wget -O "patient_joao_MICROBIOMA16S_S69_R2_001.fastq.gz" "https://aulas-pos-hiae-public-data.s3.sa-east-1.amazonaws.com/TCC-metagenomica/patient_joao_MICROBIOMA16S_S69_R2_001.fastq.gz"
wget -O "gg-13-8-99-515-806-nb-classifier.qza" "https://data.qiime2.org/2021.2/common/gg-13-8-99-515-806-nb-classifier.qza"

echo "sample-id" > sample-id.txt
echo "forward-absolute-filepath" > forward-absolute-filepath.txt
echo "reverse-absolute-filepath" > reverse-absolute-filepath.txt

ls *R1* | awk -F _ '{print $1}' >> sample-id.txt
find $PWD/*R1* >> forward-absolute-filepath.txt
find $PWD/*R2* >> reverse-absolute-filepath.txt

paste sample-id.txt forward-absolute-filepath.txt reverse-absolute-filepath.txt > manifest-file.tsv

mkdir -p fastq import dada2 visualization tree taxonomy

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

qiime tools import --type 'SampleData[PairedEndSequencesWithQuality]' --input-path manifest-file.tsv --output-path import/import.qza --input-format PairedEndFastqManifestPhred33V2
qiime demux summarize --i-data import/import.qza --o-visualization visualization/import.qzv

conda deactivate

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

qiime dada2 denoise-paired --i-demultiplexed-seqs import/import.qza \
 --p-trunc-len-f 250 \
 --p-trunc-len-r 250 \
 --p-trim-left-f 17 \
 --p-trim-left-r 21 \
 --o-representative-sequences dada2/rep-seqs.qza \
 --p-n-threads 8 \
 --o-table table.qza \
 --o-denoising-stats dada2/stats.qza

conda deactivate

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

qiime feature-table tabulate-seqs --i-data dada2/rep-seqs.qza --o-visualization visualization/rep-seqs.qzv
qiime metadata tabulate --m-input-file dada2/stats.qza --o-visualization visualization/stats.qzv

conda deactivate

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

 qiime phylogeny align-to-tree-mafft-fasttree \
 --i-sequences dada2/rep-seqs.qza \
 --o-alignment tree/aligned-rep-seqs.qza \
 --o-masked-alignment tree/masked-aligned-rep-seqs.qza \
 --o-tree tree/unrooted-tree.qza \
 --o-rooted-tree tree/rooted-tree.qza

conda deactivate

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

qiime feature-classifier classify-sklearn \
 --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
 --p-reads-per-batch 10000 \
 --i-reads dada2/rep-seqs.qza \
 --o-classification taxonomy/taxonomy.qza

conda deactivate

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

qiime metadata tabulate \
 --m-input-file taxonomy/taxonomy.qza \
 --o-visualization visualization/taxonomy.qzv

conda deactivate

In [ ]:
%%shell

eval "$(conda shell.bash hook)"
conda activate qiime2-2021.2

qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy/taxonomy.qza \
  --m-metadata-file metagen/metadata-file.tsv \
  --o-visualization visualization/taxa-bar-plots.qzv

conda deactivate

# Os resultados estão todos na pasta visualization. Usar o site https://view.qiime2.org/ para visualizar. Se tudo der certo, o arquivo taxa-bar-plots.qzv deve sair parecido com a imagem dentro da pasta metagen.